In [21]:
import pandas as pd
import numpy as np
np.random.seed(2)
from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten,  MaxPooling2D, Conv2D
from keras.callbacks import TensorBoard
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [22]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [23]:
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1)
del train 
n_classes=10

In [24]:
X_train = X_train / 255.0
test = test / 255.0

In [25]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = n_classes)
random_seed=2
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=random_seed)

In [30]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)) )
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())          
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(n_classes, activation='softmax'))

In [31]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [32]:
tensor_board = TensorBoard('./logs/LeNet-MNIST-1')

In [37]:
model.fit(X_train, Y_train, batch_size=86, epochs=15, verbose=1,validation_data=(X_val,Y_val), callbacks=[tensor_board])

Train on 37800 samples, validate on 4200 samples
Epoch 1/15
37800/37800 [==============================] - 20s 517us/step - loss: 0.0794 - acc: 0.9761 - val_loss: 0.0470 - val_acc: 0.9852
Epoch 2/15
37800/37800 [==============================] - 29s 757us/step - loss: 0.0544 - acc: 0.9835 - val_loss: 0.0444 - val_acc: 0.9871
Epoch 3/15
37800/37800 [==============================] - 43s 1ms/step - loss: 0.0442 - acc: 0.9855 - val_loss: 0.0493 - val_acc: 0.9869
Epoch 4/15
37800/37800 [==============================] - 42s 1ms/step - loss: 0.0367 - acc: 0.9882 - val_loss: 0.0415 - val_acc: 0.9874
Epoch 5/15
37800/37800 [==============================] - 42s 1ms/step - loss: 0.0310 - acc: 0.9896 - val_loss: 0.0383 - val_acc: 0.9905
Epoch 6/15
37800/37800 [==============================] - 42s 1ms/step - loss: 0.0266 - acc: 0.9913 - val_loss: 0.0500 - val_acc: 0.9881
Epoch 7/15
37800/37800 [==============================] - 42s 1ms/step - loss: 0.0244 - acc: 0.9925 - val_loss: 0.0377 - val_

In [38]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [39]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)